In [1]:
import cv2
import numpy as np
from scipy import signal

### A single edge and its DFT

In [2]:
a = np.array(np.zeros(shape=(256,256)))
a[:,128:] = 1

In [3]:
cv2.imshow("img",np.double(a))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
af = np.fft.fftshift(np.fft.fft2(a))

double형(값이 0과 1사이)로 만들어야 스펙트럼을 볼 수 있다...(?)

In [5]:
af1 = np.log(1+abs(af))
cv2.imshow("af1",af1/af1[128][128])
cv2.waitKey(0)
cv2.destroyAllWindows()

### A box and its DFT

In [6]:
a = np.zeros(shape=(256,256))
a[78:178, 78:178] = 1

In [7]:
cv2.imshow("img",np.double(a))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
af = np.fft.fftshift(np.fft.fft2(a))

In [9]:
af1 = np.log(1+abs(af))
cv2.imshow("af1",af1/af1[128][128])
cv2.waitKey(0)
cv2.destroyAllWindows()

### A rotated box and its DFT

In [10]:
x = np.linspace(0, 256, 256)
y = np.linspace(0, 256, 256)
[xv, yv] = np.meshgrid(x, y)
b = (xv+yv<329) & (xv+yv>182) & (xv-yv>-67) & (xv-yv<73)

In [11]:
cv2.imshow("b",np.double(b))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
bf = np.fft.fftshift(np.fft.fft2(b))

In [13]:
bf1 = np.log(1+abs(bf))
cv2.imshow("af1",bf1/np.max(bf1))
cv2.waitKey(0)
cv2.destroyAllWindows()

### A circle and its DFT

In [14]:
x = np.linspace(-128, 127, 256)
y = np.linspace(-128, 127, 256)
[xv, yv] = np.meshgrid(x, y)
z = np.sqrt(xv**2 + yv**2)
c = z<30

In [15]:
cv2.imshow("c",np.double(c))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
cf = np.fft.fftshift(np.fft.fft2(c))

In [17]:
cf1 = np.log(1+abs(cf))
cv2.imshow("cf1",cf1/np.max(cf1))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Ideal filtering

In [18]:
cm = cv2.imread("./rabbit.jpg", cv2.IMREAD_GRAYSCALE)
cm = cv2.resize(cm, (256,256))
img_cf = np.fft.fftshift(np.fft.fft2(cm))

In [19]:
cf1 = np.log(1+abs(img_cf))
cv2.imshow("cf1",cf1/cf1[128][128])
cv2.waitKey(0)
cv2.destroyAllWindows()

### LPF

fftshift로 중앙에 low freq components가 몰려있다 -> 중앙에 가까운 값들은 유지하고 중앙에서 먼 값은 치워버리는 matrix를 곱하자

이때, 그 matrix가 원 이다

In [20]:
x = np.linspace(-128, 127, 256)
y = np.linspace(-128, 127, 256)
[xv, yv] = np.meshgrid(x, y)
z = np.sqrt(xv**2 + yv**2)
c = z<30

In [21]:
cf = img_cf*c
cf1 = np.log(1+abs(cf))

원을 곱함으로서 중앙의 값들만 원모양으로 남았다

In [22]:
cv2.imshow("cf1",cf1/np.max(cf1))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [23]:
cfli = np.fft.ifft2(cf)
cfli = np.abs(cfli)
cv2.imshow("cfli", cfli/np.max(cfli))
cv2.waitKey(0)
cv2.destroyAllWindows()

### HPF

LPF와 반대로 중앙의 값들을 없앤다

In [24]:
x = np.linspace(-128, 127, 256)
y = np.linspace(-128, 127, 256)
[xv, yv] = np.meshgrid(x, y)
z = np.sqrt(xv**2 + yv**2)
c = z>15

In [25]:
cfh = img_cf*c
cfh1 = np.log(1+abs(cfh))

In [26]:
cv2.imshow("cfh1",cfh1/np.max(cfh1))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [27]:
cfhi = np.fft.ifft2(cfh)
cfhi = np.abs(cfhi)

In [28]:
cv2.imshow("cfhi",cfhi/np.max(cfh1))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Butterworth filtering

Ideal filter의 단점.. 경계가 미분가능하지 않기때문에 결과물에 원치않는 효과들이 생김(ringing 등등)


매끄러운 cutoff를 통해 이를 개선하자

### LPF

In [42]:
x = np.linspace(-128, 127, 256)
y = np.linspace(-128, 127, 256)
[xv, yv] = np.meshgrid(x, y)
#225로 하면 hpf가 너무 안나와서 cutoff 값 조정..
c = 1/(1+(np.sqrt(2)-1)*((xv**2+yv**2)/100)**2)

Ideal LPF와 달리 원의 경계선이 명확하지 않음

In [43]:
bl = img_cf*c
bl1 = np.log(1+abs(bl))
cv2.imshow("butterworth lpf",bl1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
bli = np.fft.ifft2(bl)
bli = np.abs(bli)

In [32]:
cv2.imshow("bli",bli/np.max(bli))
cv2.waitKey(0)
cv2.destroyAllWindows()

## HPF

In [33]:
c = 1 - c

In [34]:
bh = img_cf*c
bh1 = np.log(1+abs(bh))
cv2.imshow("butterworth lpf",bh1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [35]:
bhi = np.fft.ifft2(bh)
bhi = np.abs(bhi)

In [36]:
cv2.imshow("bli",bhi/np.max(bhi))
cv2.waitKey(0)
cv2.destroyAllWindows()

## Gaussian filter

### LPF

In [70]:
x = np.linspace(-128, 127, 256)
y = np.linspace(-128, 127, 256)
[xv, yv] = np.meshgrid(x, y)
z = np.sqrt(xv**2 + yv**2)
print(z.shape)
print((z<30).shape)
c = np.zeros(z.shape)
for i in range(256):
    for j in range(256):
        if z[i,j]<30:
            c[i,j] = z[i,j]/30
        else:
            c[i,j] = 0
                
#c = z[z<30]
print(c.shape)

(256, 256)
(256, 256)
(256, 256)


In [57]:
print(c)

[29.83286778 29.61418579 29.42787794 ... 29.42787794 29.61418579
 29.83286778]


In [72]:
c = np.log(1+abs(c))
c = cv2.GaussianBlur(c,(3,3),cv2.BORDER_DEFAULT)
cv2.imshow("gaussian blurred circle",c)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [73]:
bl = img_cf*c
bl1 = np.log(1+abs(bl))
cv2.imshow("gaussian lpf",bl1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [74]:
bli = np.fft.ifft2(bl)
bli = np.abs(bli)

In [75]:
cv2.imshow("bli",bli/np.max(bli))
cv2.waitKey(0)
cv2.destroyAllWindows()

이게 맞는건지 모르겠다...

### HPF

In [83]:
c = 1-c

In [84]:
bl = img_cf*c
bl1 = np.log(1+abs(bl))
cv2.imshow("gaussian lpf",bl1)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [85]:
bli = np.fft.ifft2(bl)
bli = np.abs(bli)

In [87]:
cv2.imshow("bli",bli/np.max(bli))
cv2.waitKey(0)
cv2.destroyAllWindows()

뭔가 아닌거같은데